# Analsisi Exploratorio de Datos FACSAT-2

Para la ejecucion del siguiente proceso de exploracion y analisis de datos, se usaron las siguientes librerias:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from pymongo import MongoClient
from IPython.display import display, Markdown

import warnings
warnings.filterwarnings('ignore')

Y adicionalmente nos apoyamos en las siguientes funciones para llevar a cabo este proceso de forma automatica:

In [2]:
# Configuración de estilo para gráficos
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Función para cargar datos desde MongoDB
def load_data(mongo_uri: str, db_name: str, collection: str, **kwargs):
    """
    Ejecuta una consulta en MongoDB con parámetros de conexión separados y opciones de consulta flexibles.
    
    Args:
        mongo_uri (str): URI de conexión a MongoDB (incluyendo credenciales si es necesario)
        db_name (str): Nombre de la base de datos
        collection (str): Nombre de la colección
        **kwargs: Argumentos opcionales para la consulta
            - query (dict): Filtro de la consulta (default: {})
            - projection (dict): Campos a proyectar (default: {})
            - skip (int): Número de documentos a saltar (default: 0)
            - limit (int): Límite de documentos a retornar (default: 0)
    
    Returns:
        DataFrame: DataFrame con los datos de la consulta
    
    Raises:
        Exception: Si hay errores en la conexión o consulta
    """
    query = kwargs.get('query', {})
    projection = kwargs.get('projection', {})
    skip = kwargs.get('skip', 0)
    limit = kwargs.get('limit', 0)
    
    try:
        client = MongoClient(mongo_uri)
        db = client[db_name]
        collection_obj = db[collection]

        result = collection_obj.find(
            query,
            projection
        ).skip(skip).limit(limit)
        
        return pd.json_normalize(list(result))

    except Exception as e:
        raise Exception(f"Error al ejecutar la consulta MongoDB: {str(e)}")

# Generar ambos gráficos en una sola figura
def plot_histogram_and_boxplot(data, col, sistema, tabla):
    # display(Markdown(f"- Graficos de la variable {col.split(f'{sistema+tabla}_')[1]}:"))
    
    # Crear una figura con dos subplots, uno al lado del otro
    fig, axes = plt.subplots(1, 2, figsize=(8, 4))

    # Histograma
    sns.histplot(data[col].dropna(), kde=True, bins=15, ax=axes[0])
    axes[0].set_title(f"Histograma - {col.split(f'{sistema+tabla}_')[1]} ({sistema} tabla {tabla})")
    axes[0].set_xlabel(col.split(f'{sistema+tabla}_')[1])

    sns.boxplot(x=data[col].dropna(), ax=axes[1])
    axes[1].set_title(f"Boxplot - {col.split(f'{sistema+tabla}_')[1]} ({sistema} tabla {tabla})")
    axes[1].set_xlabel(col.split(f'{sistema+tabla}_')[1])
    
    # Ajustar los espacios entre los subplots
    plt.tight_layout()
    
    # Mostrar la figura
    plt.show()

def crear_tabla_estadisticas(df_desc):
    """
    Crea y muestra una tabla de estadísticas descriptivas con formato mejorado.
    
    Args:
        df_desc: DataFrame con las estadísticas descriptivas
    """
    # Formatear valores numéricos para mejor legibilidad
    df_formatted = df_desc.copy()
    for columna in df_formatted.columns:
        if columna not in ['count', 'null_count']:
            df_formatted[columna] = df_formatted[columna].apply(lambda x: f"{x:.2f}")
        else:
            df_formatted[columna] = df_formatted[columna].apply(lambda x: f"{int(x)}")
    
    # Crear figura y tabla
    fig, ax = plt.subplots(figsize=(14, 1.5))
    
    # Crear tabla con mejores ajustes
    tabla = ax.table(
        cellText=df_formatted.values,
        colLabels=df_formatted.columns,
        loc='center',
        cellLoc='center',
        colWidths=[0.11] * len(df_formatted.columns)
    )
    
    # Ajustar estilo de la tabla
    tabla.auto_set_font_size(False)
    tabla.set_fontsize(12)
    tabla.scale(1, 1.8)  # Aumentar altura de las celdas
    
    # Estilizar celdas
    for k, cell in tabla._cells.items():
        cell.set_edgecolor('black')
        if k[0] == 0:  # Encabezados
            cell.set_text_props(weight='bold')
            cell.set_facecolor('#e6e6e6')
            cell.set_fontsize(14)
    
    # Ajustes finales de la figura
    ax.axis('off')
    plt.tight_layout()
    plt.show()

# Análisis univariado para cada sistema y tabla
def univariate_analysis(data, system_name):
    # Configuración general de matplotlib
    try:
        import seaborn as sns
        plt.style.use('seaborn')
    except:
        # Si seaborn no está disponible, usar estilo básico de matplotlib
        plt.style.use('default')

    plt.rcParams['font.size'] = 10
    plt.rcParams['figure.facecolor'] = 'white'
    # plt.rcParams['axes.facecolor'] = 'white'

    # Extraer nombre del sistema y tabla
    sistema = str(system_name.split("_")[0])
    tabla = str(system_name.split("_")[1])

    # Mostrar título del sistema
    display(Markdown(f"### Sistema {system_name}:"))
    
    # Distribución de variables numéricas
    for col in data.select_dtypes(include=[np.number]).columns:
        display(Markdown(f"### Análisis de la variable `{col.split(f'{sistema+tabla}_')[1]}`"))
        display(Markdown(f"#### Estadisticas descriptivas:"))
        # Estadísticas descriptivas
        df_desc = pd.DataFrame(data[col].describe())
        df_desc.loc['null_count'] = data[col].isnull().sum()
        df_desc = df_desc.T

        # Crear y mostrar la tabla
        crear_tabla_estadisticas(df_desc)

        # Crear una figura con dos subplots
        display(Markdown(f"#### Graficos descriptivos:"))
        plot_histogram_and_boxplot(data, col, sistema, tabla)

# Análisis multivariado entre variables de diferentes sistemas
def multivariate_analysis(merged_df):
    print(f"\n--- Análisis multivariado (sistemas combinados) ---")
    
    # Matriz de correlación
    corr_matrix = merged_df.corr()
    sns.heatmap(corr_matrix, annot=False, cmap="coolwarm", fmt=".2f")
    plt.title("Matriz de correlación entre variables de diferentes sistemas")
    plt.show()
    
    # PCA para reducir la dimensionalidad y visualizar patrones
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(merged_df.dropna())
    pca = PCA(n_components=2)
    pca_data = pca.fit_transform(scaled_data)
    
    plt.scatter(pca_data[:, 0], pca_data[:, 1], alpha=0.5)
    plt.title("PCA - Componentes principales de los datos combinados de sistemas")
    plt.xlabel("Componente Principal 1")
    plt.ylabel("Componente Principal 2")
    plt.show()
    
    return corr_matrix, pca_data

# Detección de anomalías con Z-score
def zscore_anomaly_detection(merged_df, threshold=3):
    z_scores = np.abs(zscore(merged_df.dropna()))
    anomalies = (z_scores > threshold).any(axis=1)
    
    print(f"Anomalías detectadas: {np.sum(anomalies)}")
    
    return merged_df[anomalies], anomalies

# Función principal para ejecutar el análisis completo
def main():

    # Configuración de conexión a MongoDB
    mongo_uri = "mongodb://localhost:27017/"
    db_name = "etl_data"
    
    client = MongoClient(mongo_uri)
    db = client[db_name]

    collections = db.list_collection_names()  # Lista de colecciones por sistema
    
    # Cargar datos de cada sistema
    data_frames = []
    for collection in collections:
        # Cargar datos
        data = load_data(mongo_uri=mongo_uri, db_name=db_name, collection=collection)
        data_frames.append(data)
        
        # # Análisis Univariado
        # # Insertar un título de Markdown
        # display(Markdown("## Análisis Univariado"))
        # univariate_analysis(data, collection)
        
    # Fusión de datos para el análisis multivariado y la detección de anomalías
    merged_df = pd.concat(data_frames, axis=1, join="inner")

    # Insertar un subtítulo para el siguiente análisis
    display(Markdown("## Análisis Multivariado"))
    corr_matrix, pca_data = multivariate_analysis(merged_df)
    
    # Detección de anomalías
    anomalies, anomaly_flags = zscore_anomaly_detection(merged_df)

    return {
        "correlation_matrix": corr_matrix,
        "pca_data": pca_data,
        "anomalies": anomalies,
    }

Para llevar a cabo la ejecucion del proceso automatico, se debe ejecutar la siguiente funcion principal:

In [4]:
# Llamada principal
if __name__ == "__main__":
    eda_results = main()

## Análisis Multivariado


--- Análisis multivariado (sistemas combinados) ---


TypeError: float() argument must be a string or a real number, not 'ObjectId'